In [9]:
# from open ai cookbook:
# https://cookbook.openai.com/examples/assistants_api_overview_python

from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [10]:
#step 1 - create an assistant
assistant = client.beta.assistants.create(
    name = "math tutor",
    instructions = "You are a personal math tutor. Write and run code to answer math questions.",
    tools = [{"type": "code_interpreter"}],
    model = "gpt-3.5-turbo"
)

In [11]:
#step 2 - create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_ChaL5tw1Tf7iv6Nr9vvpVVjS', created_at=1719937686, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [12]:
#step 3 - add a message to the thread
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "Solve this problem: 3x + 11 = 14"
)
print(message)

In [13]:
#step 4 - run the assistant
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)
print(run)

In [14]:
#step 5 - display the assistant's response
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id, 
    run_id = run.id
)
print(run)

In [15]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)
print(messages)

In [16]:
for message in reversed(messages.data): 
    print(message.role + ": " + message.content[0].text.value)

user: Solve this problem: 3x + 11 = 14
assistant: To solve the equation \( 3x + 11 = 14 \) for \( x \), follow these steps:

1. Subtract 11 from both sides of the equation to isolate the term containing \( x \).
2. Divide both sides by 3 to solve for \( x \).

Let's perform these calculations.
